In [8]:
import pprint
import IPython.display as ipd
import torch
import librosa

In [2]:
# load model
mars5, config_class = torch.hub.load('Camb-ai/mars5-tts', 'mars5_english', trust_repo=True)

/home/ack/miniconda3/envs/matt-py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/ack/miniconda3/envs/matt-py311/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/home/ack/miniconda3/envs/matt-py311/lib/python3.11/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Now that the model is loaded, pick a reference audio to clone from. If you want to use deep clone, also specify its transcript. 

In [13]:
# download example ref audio
!wget -O example.flac https://files.catbox.moe/6i7a4n.flac 

--2024-06-05 14:30:47--  https://files.catbox.moe/6i7a4n.flac
Resolving files.catbox.moe (files.catbox.moe)... failed: Name or service not known.
wget: unable to resolve host address ‘files.catbox.moe’


In [13]:
wav, sr = librosa.load('./example.flac', 
                       sr=mars5.sr, mono=True)
wav = torch.from_numpy(wav)
ref_transcript = "We actually haven't managed to meet demand."
print("Reference audio:")
ipd.display(ipd.Audio(wav.numpy(), rate=mars5.sr))
print(f"Reference transcript: {ref_transcript}")

Reference audio:


Reference transcript: Ask her to bring these things with her from the store


In [15]:
deep_clone = True # set to False if you don't know prompt transcript or want fast inference.
# Below you can tune other inference settings, like top_k, temperature, top_p, etc...
cfg = config_class(deep_clone=deep_clone, rep_penalty_window=100,
                      top_k=100, temperature=0.7, freq_penalty=3)

ar_codes, wav_out = mars5.tts("the quick brown rat.", wav, 
          ref_transcript,
          cfg=cfg)

print('Synthesized output audio:')
ipd.Audio(wav_out.numpy(), rate=mars5.sr)

Note: using deep clone. Assuming input `c_phones` is concatenated prompt and output phones. Also assuming no padded indices in `c_codes`.
New x: torch.Size([1, 705, 8]) | new x_known: torch.Size([1, 705, 8]) . Base prompt: torch.Size([1, 299, 8]). New padding mask: torch.Size([1, 705]) | m shape: torch.Size([1, 705, 8])
Synthesized output audio:


You can see all the inference settings available to tune in the inference config here:

In [12]:
pprint.pprint(config_class())

InferenceConfig(temperature=0.7,
                top_k=200,
                top_p=0.2,
                typical_p=1.0,
                freq_penalty=3,
                presence_penalty=0.4,
                rep_penalty_window=80,
                eos_penalty_decay=0.5,
                eos_penalty_factor=1,
                eos_estimated_gen_length_factor=1.0,
                timesteps=200,
                x_0_temp=0.7,
                q0_override_steps=20,
                nar_guidance_w=3,
                max_prompt_dur=12,
                generate_max_len_override=-1,
                deep_clone=True,
                use_kv_cache=True,
                trim_db=27,
                beam_width=1,
                ref_audio_pad=0)


You can also listen to the vocoded raw coarse codes, for debugging purposes:

In [8]:
ar_wav = mars5.vocode(ar_codes.cpu()[:, None])
ipd.Audio(ar_wav.numpy(), rate=mars5.sr)